# Importing needed libraries

In [1]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import date

In [2]:
# !pip install selenium

In [3]:
# For scraping
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import requests
from selenium import webdriver

# Loading the datasets

In [4]:
df_industries = pd.read_csv('./Raw Data/industries.csv', delimiter= ';')
df_companies = pd.read_csv('./Raw Data/us-companies.csv', delimiter= ';')
df_balance = pd.read_csv('./Raw Data/us-balance-quarterly.csv', delimiter= ';')
df_cashflow = pd.read_csv('./Raw Data/us-cashflow-quarterly.csv', delimiter= ';')
# df_shareprices = pd.read_csv('./Raw Data/us-shareprices-daily.csv', delimiter= ';')

# EDA & Cleaning

# Industries

In [5]:
display(df_industries.head())
display(df_industries.isna().sum())
display(df_industries.shape)

,IndustryId,Sector,Industry
0,100001,Industrials,Industrial Products
1,100002,Industrials,Business Services
2,100003,Industrials,Engineering & Construction
3,100004,Industrials,Waste Management
4,100005,Industrials,Industrial Distribution


IndustryId    0
Sector        0
Industry      0
dtype: int64

(74, 3)

Defining the industry and sector according to an ID, all data seems to be complete and clean

# Companies

## First look

In [6]:
display(df_companies.head(3))
display(df_companies.isna().sum())
display(df_companies.shape)

,Ticker,SimFinId,Company Name,IndustryId
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0
1,A18,1253413,Trip.com Group Ltd,NaN
2,A21,1333027,Li Auto Inc.,NaN


Ticker            0
SimFinId          0
Company Name      0
IndustryId      416
dtype: int64

(3144, 4)

## There is data missing for the industry ID's

In [7]:
# It seems that for some companies we do not have the indusry data for a lot of companies, we investiate.
df_companies[df_companies['IndustryId'].isna()].head(5)

,Ticker,SimFinId,Company Name,IndustryId
1,A18,1253413,Trip.com Group Ltd,NaN
2,A21,1333027,Li Auto Inc.,NaN
21,ABEO,953884,ABEONA THERAPEUTICS INC.,NaN
23,ABIO,953886,"ARCA biopharma, Inc.",NaN
34,ABX,1271527,BARRICK GOLD CORP,NaN


Apparently the missing values are ranging from a very diverse set of industries.
I try to assign some of the missing values by deductingh from the company names.

## Tokenize
To better adress the different companies by name, I tokenize them

In [8]:
# We tokenize the company names, so we can search through all appearing words
from nltk.tokenize import word_tokenize

In [9]:
df_companies['tokens'] = df_companies['Company Name'].apply(word_tokenize)

In [10]:
df_companies.head(5)

,Ticker,SimFinId,Company Name,IndustryId,tokens
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0,"[AGILENT, TECHNOLOGIES, INC]"
1,A18,1253413,Trip.com Group Ltd,NaN,"[Trip.com, Group, Ltd]"
2,A21,1333027,Li Auto Inc.,NaN,"[Li, Auto, Inc, .]"
3,AA,367153,Alcoa Corp,110004.0,"[Alcoa, Corp]"
4,AAC_delist,939324,"AAC Holdings, Inc.",106011.0,"[AAC, Holdings, ,, Inc, .]"


## Functions for df_companies

In [11]:
# A function that returns one if one of our keywords is found
def finder(tokens):
    for i in tokens:
        if i.lower() in keywords:
            return 1
    return 0   

In [12]:
# A function, that displays the rows which contain keywords an have no industry ID 
# It also return a list of the corresponding indexes.

def nan_list(df,keywords):
    # Finding the companies with keywords
    df['current_search'] = 0
    df['current_search'] = df['tokens'].apply(finder)
    
    # Displaying them if they have no industrty-id
    display(df[ (df['current_search']==1) & (df['IndustryId'].isna()) ])
    
    # Saving the indexes
    indexlist = list(df[ (df['current_search']==1) & (df['IndustryId'].isna()) ].index)
    
    return indexlist

In [13]:
def set_industryID(industry, df, indexlist):
    
    # Getting the ID according to industry
    id = df_industries[df_industries['Industry']==industry]['IndustryId'].unique()[0]  
         
    # Setting the id for the specified indexes
    for row in indexlist:
        df.iloc[row,3] = id
    
    return df    

## ETF's & Funds

Since ETFs Funds and REITs do not usually issue debt notes or they have to be evaluated completely different, I will ecxlude them from this analysis.

In [14]:
keywords = ['etf','etc','fund','reit','trust']
df_companies['current_search'] = 0
df_companies['current_search'] = df_companies['tokens'].apply(finder)
# display(df_companies[df_companies['current_search']==1])
len(df_companies[df_companies['current_search']==1])

52

In [15]:
df_companies.shape

(3144, 6)

In [16]:
# We drop the according rows
df_companies = df_companies[df_companies['current_search'] == 0]
df_companies.shape

(3092, 6)

## Scraping the missing data
Apparently nasdaq loads the sites content ad hoc using java scripts and normal get request won't work, therfore we use selenium to mimic a real browser access

In [17]:
# Importing the webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By

In [18]:
# To wait until the finder finds the expected element, sicne it takes some time to load
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [19]:
# Defning the path of the chromedriver
path = 'D:/Chromium/chromedriver'

In [20]:
# Setting up driver
driver = webdriver.Chrome(path)

C:\Users\Giancarlo\AppData\Local\Temp\ipykernel_13960\2479737333.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [21]:
# Telling the driver to wait for up to 20 seconds if element is not found
wait = WebDriverWait(driver, 20)

#### Create alist of company tickers with missing values

In [22]:
ticker_list = list(df_companies[df_companies['IndustryId'].isna()]['Ticker'])
len(ticker_list)

394

In [23]:
# Testing with wait function
# ticker = 'DOW'
# target_url = f"https://finance.yahoo.com/quote/%s/profile?p=%s" % (ticker,ticker)
# target_url

In [24]:
# driver.get(target_url)
# element = wait.until(EC.presence_of_element_located(('xpath', '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]')))
# element.text

#### Scraping the missing data

Takes quite some time, so I saved the results as a pickle

In [25]:
# sectors = []
# for ticker in ticker_list:
#     # Setting url
#     target_url = f"https://finance.yahoo.com/quote/%s/profile?p=%s" % (ticker,ticker)
#     # Accessing the site
#     driver.get(target_url)    

#     # Getting the desired field:
#     try:
#         match = wait.until(EC.presence_of_element_located(('xpath', '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]')))
#         sectors.append(match.text)
#         print(match.text)
#     except:
#         sectors.append('Not found')
#         print('Not found')
#     print(len(sectors))

# # Closing window
# driver.quit()

In [26]:
# # Saving Sectorlist
import pickle
# pickle.dump(sectors, open('sectors.p', 'wb'))

In [27]:
sectors = pickle.load(open('sectors.p','rb'))
# sectors

#### Assigning the scraped data to the company dataframe

In [28]:
# We translate the industry names to industry ID's
sectors_id = []
for i in range(len(sectors)):
    try:
        sectors_id.append(int(df_industries[df_industries['Industry'] == sectors[i]]['IndustryId']))
    except:
        # If the industry is not found, we append the Industry ID for 'Other'
        sectors_id.append('108004')

In [29]:
# We assign the industry names to a new column
df_companies.loc[df_companies['IndustryId'].isna(), ['IndustryId']] = sectors_id

In [30]:
# We got rid of all NaN's
df_companies['IndustryId'].isna().sum()

0

In [31]:
# We save the cleaned dataframe to our processed Data
df_companies.to_csv('Prepared Data/company_data.csv', index=False)

# Balance sheets

In [32]:
df_balance = pd.read_csv('./Raw Data/us-balance-quarterly.csv', delimiter= ';')

In [33]:
display(df_balance.head(1))

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,A,45846,USD,2017,Q1,2017-01-31,2017-03-08,2017-03-08,322000000.0,326000000.0,...,190000000.0,1089000000,1.802000e+09,2.483000e+09,3572000000,5.239000e+09,0.0,-453000000.0,4.300000e+09,7872000000


## Dropping unneccessary columns

In [34]:
# We can already see, that there are several column that we won't need, so we drop them first:
# SimFinID : A site specific identifier
# Currency : We have everything in USD
# We will only need the Report Date if the dates
# The share count we decide to only used the Diluted since it better represents reality
# Treasury Stock is insignificant
# Total Liabilities and equity doesn't give us any information, that we don't have otheriwse
# Total Current Assets and Total assets are redundant

In [35]:
df_balance = df_balance.drop(['SimFinId','Currency','Publish Date','Restated Date','Shares (Basic)','Treasury Stock','Total Liabilities & Equity'], axis = 1)
df_balance.head(1)

,Ticker,Fiscal Year,Fiscal Period,Report Date,Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",...,Total Assets,Payables & Accruals,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Retained Earnings,Total Equity
0,A,2017,Q1,2017-01-31,326000000.0,2.241000e+09,653000000.0,551000000.0,3.635000e+09,653000000.0,...,7872000000,268000000.0,190000000.0,1089000000,1.802000e+09,2.483000e+09,3572000000,5.239000e+09,-453000000.0,4.300000e+09


## Checking for NaN's

In [36]:
df_balance.isna().sum()

Ticker                                                 0
Fiscal Year                                            0
Fiscal Period                                          0
Report Date                                            0
Shares (Diluted)                                     297
Cash, Cash Equivalents & Short Term Investments      143
Accounts & Notes Receivable                         4857
Inventories                                        15615
Total Current Assets                                   2
Property, Plant & Equipment, Net                     945
Long Term Investments & Receivables                32101
Other Long Term Assets                               454
Total Noncurrent Assets                               91
Total Assets                                           0
Payables & Accruals                                  158
Short Term Debt                                    16510
Total Current Liabilities                              0
Long Term Debt                 

We will have to check column by column

In [37]:
df_balance[df_balance['Shares (Diluted)'].isna()]['Ticker'].unique()

array(['AGI', 'ALTMS', 'AMHCU', 'APU', 'ARPO', 'AUDH', 'BCAB', 'BKH',
       'BLUD', 'BPL', 'CCFI', 'CCH', 'CERS', 'CHAQ', 'CK00015503',
       'CK00015844', 'CK00015847', 'CNNX', 'DWACU', 'ENBL', 'EQ', 'FLUG',
       'FSII', 'IIVI', 'INFOR', 'NKT.CO', 'PROG', 'QVC', 'SFY', 'SRNGU',
       'TIVC', 'TM-28', 'VPCC', 'VTYX'], dtype=object)

In [38]:
# We see that the NaN's without share counts are split up into just 36 companies, we have a look, whart they are:
companylist = list(df_balance[df_balance['Shares (Diluted)'].isna()]['Ticker'].unique())

In [39]:
namelist = []
for company in companylist:
    try:
        namelist.append(str(df_companies[df_companies['Ticker']==company]['Company Name']))
    except:
        namlist.append(str(company)+' not found in companyframe')
namelist

# My guess, that we here have Funds that we won't include in our analysis did not hold.

['95    Affinion Group, Inc.\nName: Company Name, dtype: object',
 '158    Alta Mesa Holdings, LP\nName: Company Name, dtype: object',
 '174    Jasper Therapeutics, Inc.\nName: Company Name, dtype: object',
 '222    AMERIGAS PARTNERS LP\nName: Company Name, dtype: object',
 '245    Aadi Bioscience, Inc.\nName: Company Name, dtype: object',
 '276    AURORA DIAGNOSTICS HOLDINGS LLC\nName: Company Name, dtype: object',
 '334    BioAtla, Inc.\nName: Company Name, dtype: object',
 '386    BLACK HILLS POWER INC\nName: Company Name, dtype: object',
 '403    IMMUCOR INC\nName: Company Name, dtype: object',
 '424    BUCKEYE PARTNERS, L.P.\nName: Company Name, dtype: object',
 '520    Community Choice Financial Inc.\nName: Company Name, dtype: object',
 '521    Cheniere Corpus Christi Holdings, LLC\nName: Company Name, dtype: object',
 '554    CERUS CORP\nName: Company Name, dtype: object',
 '572    Renovacor, Inc.\nName: Company Name, dtype: object',
 '608    BakerCorp International, Inc.\nName

In [40]:
# We check if all values are NAN for the share count
for company in companylist:
    print(len(df_balance[(df_balance['Ticker'] == company) & (df_balance['Shares (Diluted)'].isna())]) == len(df_balance[(df_balance['Ticker'] == company)]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [41]:
# Since we have no information about the sharecount for these companies and there are a lot of other values missing for them,
# I decide to drop them since they are just 36 out of over 3000
print(df_balance.shape)
for company in companylist:
    df_balance = df_balance[~(df_balance['Ticker']==company)]
df_balance.shape

(42894, 23)


(42597, 23)

#### Systematic filling with mean value

Since we have many columns with a lot of missing valkues, wich make up more than 30% of our dataframe, and filling it with arbitrary values would cause a lot of error, we will drop them for now.

In [42]:
df_balance = df_balance.drop(['Inventories','Long Term Investments & Receivables','Short Term Debt','Long Term Debt'],axis = 1)
df_balance.shape

(42597, 19)

In [43]:
df_balance.isna().sum()

Ticker                                                0
Fiscal Year                                           0
Fiscal Period                                         0
Report Date                                           0
Shares (Diluted)                                      0
Cash, Cash Equivalents & Short Term Investments     136
Accounts & Notes Receivable                        4781
Total Current Assets                                  2
Property, Plant & Equipment, Net                    906
Other Long Term Assets                              431
Total Noncurrent Assets                              89
Total Assets                                          0
Payables & Accruals                                 138
Total Current Liabilities                             0
Total Noncurrent Liabilities                        672
Total Liabilities                                     0
Share Capital & Additional Paid-In Capital          600
Retained Earnings                               

In [44]:
# The other columns, will fill the NaN's with the mean value for the corresponding company
def get_mean(company,column):
    return np.mean(df_balance[ (df_balance['Ticker']==company) & (df_balance[column].isna() == False) ][column])

In [45]:
# Columns to iterate thorugh
columns = ['Cash, Cash Equivalents & Short Term Investments','Accounts & Notes Receivable','Total Current Assets','Property, Plant & Equipment, Net',
          'Other Long Term Assets','Total Noncurrent Assets','Payables & Accruals','Total Noncurrent Liabilities','Share Capital & Additional Paid-In Capital',
          'Retained Earnings','Total Equity']

# Getting a list with all company tickers
companylist = list(df_balance['Ticker'].unique())

In [ ]:
# setting mean for missing NaN
for column in columns:
    print(column)
    for company in companylist:
        df_balance.loc[ (df_balance[column].isna()) & (df_balance['Ticker']==company), [column]] = get_mean(company,column)

Cash, Cash Equivalents & Short Term Investments
Accounts & Notes Receivable
Total Current Assets


In [ ]:
df_balance.isna().sum()

In [ ]:
# Apparently there are coulumns where no mean could be calculated. We see wich comnpanies are affected.
affected = []
for column in columns:
    affected.append(df_balance[df_balance[column].isna()]['Ticker'].unique())
# Making a list of the single entries instead of an array
affected_uniques = []
for i in range(len(affected)):
    for x in affected[i]:
        affected_uniques.append(x)
# Removing duplicates
affected_uniques = list(set(affected_uniques))
len(affected_uniques)

There are 336 companies affected for which we cannot get a value. We will have to drop them for now.

In [ ]:
display(df_cashflow.head(1))
display(df_cashflow.tail(1))

In [ ]:
column = columns[0]
company = companylist[0]
df_balance[ (df_balance['Ticker']==company) & (df_balance[column].isna() == True) ][column]

In [ ]:
company

In [ ]:
company = companylist[4]
df_balance[df_balance['Ticker']==company]